In [20]:
import re

import pandas as pd
import numpy as np

In [22]:
df = pd.read_csv(r'C:\Users\nguye\Documents\GitHub\spark_shared_repo\group_02\Marvel_DC_imdb.csv')

# Drop unnamed column
df = df.drop(columns=['Unnamed: 0'])



# Clean and Format Data

In [10]:
# Fix Numericals
def str_to_float(cell, remove = ["M", "min", "$", ","]):
    if isinstance(cell, str):
        for r in remove:
            cell = cell.replace(r, "")
        return float(cell)
    else:
        return cell


df['RunTime'] = df['RunTime'].apply(str_to_float)
df['USA_Gross'] = df['USA_Gross'].apply(str_to_float)
df['Votes'] = df['Votes'].apply(str_to_float)


df


,Movie,Year,Genre,RunTime,Rating,Director,Actor,Description,IMDB_Score,Metascore,Votes,USA_Gross,Category
0,Eternals,(2021),"Action,Adventure,Drama",NaN,NaN,ChloéZhao,"AngelinaJolie,GemmaChan,RichardMadden,BarryKeo...","The saga of the Eternals, a race of immortal b...",NaN,NaN,NaN,NaN,Marvel
1,Loki,(2021– ),"Action,Adventure,Fantasy",NaN,NaN,NaN,"TomHiddleston,OwenWilson,SophiaDiMartino,Richa...",A new Marvel chapter with Loki at its center.,NaN,NaN,NaN,NaN,Marvel
2,The Falcon and the Winter Soldier,(2021),"Action,Adventure,Drama",50.0,TV-14,NaN,"AnthonyMackie,SebastianStan,WyattRussell,ErinK...","Following the events of 'Avengers: Endgame,' S...",7.5,NaN,105557.0,NaN,Marvel
3,WandaVision,(2021),"Action,Comedy,Drama",350.0,TV-PG,NaN,"ElizabethOlsen,PaulBettany,KathrynHahn,Teyonah...",Blends the style of classic sitcoms with the M...,8.1,NaN,174710.0,NaN,Marvel
4,Spider-Man: No Way Home,(2021),"Action,Adventure,Sci-Fi",NaN,NaN,JonWatts,"AngourieRice,TomHolland,Zendaya,MarisaTomei",A continuation of Spider-Man: Far From Home.,NaN,NaN,NaN,NaN,Marvel
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1685,DC's Legends of Tomorrow,(2016– ),"Action,Adventure,Drama",42.0,TV-14,GregorySmith,"BrandonRouth,CaityLotz,MaisieRichardson-Seller...","Worlds lived, worlds died. Nothing will ever b...",8.5,NaN,2050.0,NaN,DC
1686,Supergirl,(2015–2021),"Action,Adventure,Drama",42.0,TV-PG,CarlSeaton,"MelissaBenoist,MehcadBrooks,ChylerLeigh,KatieM...","In the wake of Lex Luthor's return, the show f...",8.3,NaN,1259.0,NaN,DC
1687,Supergirl,(2015–2021),"Action,Adventure,Drama",42.0,TV-PG,AlexisOstrander,"MelissaBenoist,MehcadBrooks,ChylerLeigh,KatieM...",Kara comes face to face with Red Daughter and ...,8.1,NaN,1053.0,NaN,DC
1688,Supergirl,(2015–2021),"Action,Adventure,Drama",42.0,TV-PG,ShannonKohli,"MelissaBenoist,MehcadBrooks,ChylerLeigh,KatieM...",Kara and Lena head to Kaznia to hunt down Lex....,7.4,NaN,1036.0,NaN,DC


In [11]:
# Get start and end years as int

def get_start_end_year(s):
    if isinstance(s, (float, int)):
        return s, np.nan
    s = s.replace("(", "")
    s = s.replace(")", "")
    years = re.findall(r'\d+', s)
    if len(years) == 1:
        return int(years[0]), np.nan
    elif len(years) == 2:
        return int(years[0]), int(years[1])

df[["Year_Start", "Year_End"]] = pd.DataFrame(df['Year'].apply(get_start_end_year).tolist(), index=df.index)




In [12]:
# Strip leading and trailing spaces 

str_columns = ['Movie', 'Genre', 'Director', 'Actor', 'Description',]

def strip_spaces(cell):
    if isinstance(cell, str):
        return cell.strip()
    else:
        return cell

for col in str_columns:
    df[col] = df[col].apply(strip_spaces)


In [13]:
# Get information it a series or movie (repeat instances)

count_data = df.groupby(['Movie', 'Year'])['Year_Start'].count()

movie_or_series = []

for _, row in df.iterrows():
    movie = row['Movie']
    year = row['Year']
    try:
        if count_data.loc[movie, year] == 1:
            movie_or_series.append('Movie')
        else:
            movie_or_series.append('Series')
    except:
        movie_or_series.append('Unknown')

df["Type"] = movie_or_series


df = df.dropna(subset=["Movie", "Year", "IMDB_Score"])
# Drop video games
df = df.query("~ Year.str.contains('Game')", engine="python")

In [18]:
df.set_index('Movie').to_csv(r"Marvel_DC_imdb_cleaned.csv")

In [17]:
df.set_index('Movie')

,Year,Genre,RunTime,Rating,Director,Actor,Description,IMDB_Score,Metascore,Votes,USA_Gross,Category,Year_Start,Year_End,Type
Movie,,,,,,,,,,,,,,,
The Falcon and the Winter Soldier,(2021),"Action,Adventure,Drama",50.0,TV-14,NaN,"AnthonyMackie,SebastianStan,WyattRussell,ErinK...","Following the events of 'Avengers: Endgame,' S...",7.5,NaN,105557.0,NaN,Marvel,2021.0,NaN,Series
WandaVision,(2021),"Action,Comedy,Drama",350.0,TV-PG,NaN,"ElizabethOlsen,PaulBettany,KathrynHahn,Teyonah...",Blends the style of classic sitcoms with the M...,8.1,NaN,174710.0,NaN,Marvel,2021.0,NaN,Series
Avengers: Endgame,(2019),"Action,Adventure,Drama",181.0,PG-13,"AnthonyRusso,JoeRusso","RobertDowneyJr.,ChrisEvans,MarkRuffalo,ChrisHe...",After the devastating events of Avengers: Infi...,8.4,78.0,880911.0,858.37,Marvel,2019.0,NaN,Movie
Guardians of the Galaxy,(2014),"Action,Adventure,Comedy",121.0,PG-13,JamesGunn,"ChrisPratt,VinDiesel,BradleyCooper,ZoeSaldana",A group of intergalactic criminals must pull t...,8.0,76.0,1066222.0,333.18,Marvel,2014.0,NaN,Movie
Spider-Man: Far from Home,(2019),"Action,Adventure,Sci-Fi",129.0,PG-13,JonWatts,"TomHolland,SamuelL.Jackson,JakeGyllenhaal,Mari...",Following the events of Avengers: Endgame (201...,7.5,69.0,348047.0,390.53,Marvel,2019.0,NaN,Movie
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DC's Legends of Tomorrow,(2016– ),"Action,Adventure,Drama",42.0,TV-14,GregorySmith,"BrandonRouth,CaityLotz,MaisieRichardson-Seller...","Worlds lived, worlds died. Nothing will ever b...",8.5,NaN,2050.0,NaN,DC,2016.0,NaN,Series
Supergirl,(2015–2021),"Action,Adventure,Drama",42.0,TV-PG,CarlSeaton,"MelissaBenoist,MehcadBrooks,ChylerLeigh,KatieM...","In the wake of Lex Luthor's return, the show f...",8.3,NaN,1259.0,NaN,DC,2015.0,2021.0,Series
Supergirl,(2015–2021),"Action,Adventure,Drama",42.0,TV-PG,AlexisOstrander,"MelissaBenoist,MehcadBrooks,ChylerLeigh,KatieM...",Kara comes face to face with Red Daughter and ...,8.1,NaN,1053.0,NaN,DC,2015.0,2021.0,Series
